# 支持向量机


# 例7.2 已知正例点$x_1 = (1, 2) ^T, x_2 = (2, 3)^T, x_3 = (3, 3)^T$, 负例点$x_4 = (2, 1)^T, x_5 = (3, 2)^T$

In [9]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# SVM
class SVM:
    """
    SVM 类
    """
    
    def __init__(self, trainDataList, trainLabelList, sigma = 10, C = 200, tol = 0.001):
        """
        SVM相关参数初始化
        sigma : 高斯核中的分母sigma
        C : 软间隔中的惩罚参数
        tol ：松弛变量
        """
        
        self.trainDataMat = np.mat(trainDataList)
        self.trainLabelMat = np.mat(trainLabelList).T
        
        self.m, self.n = np.shape(self.trainDataMat)  # m 样本数量 n 特征个数
        self.sigma = sigma
        self.C = C
        self.tol = tol
        
        self.k = self.calcKernel()  # 核函数
        self.b = 0
        self.alpha = [0] * self.trainDataMat.shape[0]
        self.E = [0 * self.trainLabelMat[i, 0] for i in range(self.trainLabelMat.shape[0])]  #SMO 运算过程中的Ei
        self.supportVecIndex = []
        
        
    def calcKernel(self):
        """
        计算核函数 高斯核函数
        """
        
        # 初始化高斯矩阵 k[i][j] = Xi * Xj
        k = [[0 for i in range(self.m)] for j in range(self.m)]
        
        for i in range(self.m):
            if i % 100 == 0:
                print("construct the kernel: ", i, self.m)
            
            X = self.trainDataMat[i, :]
            #
            for j in range(i, self.m):
                Z = self.trainDataMat[j, :]
                result = (X - Z) * (X - Z).T
                result = np.exp(-1 * result / (2 * self.sigma ** 2))
                k[i][j] = result
                k[j][i] = result
                
        return k
    
    
    def isSatifyKKT(self, i):
        """
        查看第i个alpha是否满足KKT条件
        """
        
        gxi = self.calc_gxi(i)
        yi = self.trainLabelMat[i]
        
        if (math.fabs(self.alpha[i]) < self.tol) and (yi * gxi >= 1):
            return True
        elif (math.fabs(self.alpha[i] - self.C) < self.tol) and (yi * gxi <= 1):
            return True
        elif (self.alpha[i] > -self.tol) and (self.alpha[i] < (self.C + self.tol)) and (math.fabs(yi * gxi - 1) < self.tol):
            return True
        
        return False
    
    def calc_gxi(self, i):
        """
        计算g(xi)
        """
        
        gxi = 0
        index = [i for i, alpha in enumerate(self.alpha) if alpha != 0]
        
        for j in index:
            gxi += self.alpha[j] * self.trainLabelMat[j] * self.k[j][i]
        
        gxi += self.b
        
        return gxi
    
    def calc_Ei(self, i):
        """
        计算Ei
        """
        gxi = self.calc_gxi(i)
        return gxi - self.trainLabelMat[i]
        
    def getAlphaJ(self, E1, i):
        """
        SMO 选择第二个变量
        """
        
        E2 = 0
        
        maxE1_E2 = -1
        maxIndex = -1
        
        nozeroE = [i for i, Ei in enumerate(self.E) if Ei != 0]
        
        for j in nozeroE:
            E2_tmp = self.calc_Ei(j)
            if math.fabs(E1 - E2_tmp) > maxE1_E2:
                maxE1_E2 = math.fabs(E1 - E2_tmp)
                E2 = E2_tmp
                maxIndex = j
        
        if maxIndex == -1:
            maxIndex = i
            while maxIndex == i:
                maxIndex = int(random.uniform(0, self.m))
            E2 = self.calc_Ei(maxIndex)
            
        return E2, maxIndex
    
    def train(self, max_iter = 100):
        """
        训练
        """
        iterStep = 0
        paramterChanged = 1
        
        while (iterStep < max_iter) and (paramterChanged > 0):
            print('iter: %d: %d' % (iterStep, max_iter))
            iterStep += 1
            paramterChanged = 0
            
            for i in range(self.m):
                if self.isSatifyKKT(i) == False:
                    E1 = self.calc_Ei(i)
                    
                    E2, j = self.getAlphaJ(E1, i)
                    
                    y1 = self.trainLabelMat[i]
                    y2 = self.trainLabelMat[j]
                    
                    alpha0ld_1 = self.alpha[i]
                    alpha0ld_2 = self.alpha[j]
                    
                    if y1 != y2:
                        L = max(0, alpha0ld_2 - alpha0ld_1)
                        H = min(self.C, self.C + alpha0ld_2 - alpha0ld_1)
                    else:
                        L = max(0, alpha0ld_2 + alpha0ld_1 - self.C)
                        H = min(self.C, alpha0ld_2 + alpha0ld_1)
                        
                    if L == H:
                        continue
                        
                    k11 = self.k[i][i]
                    k12 = self.k[i][j]
                    k21 = self.k[j][i]
                    k22 = self.k[j][j]
                    
                    alphaNew_2 = alpha0ld_2 + y2 * (E1 - E2) / (k11 + k22 - 2 * k12)
                    
                    if alphaNew_2 < L :
                        alphaNew_2 = L
                    elif alphaNew_2 > H:
                        alphaNew_2 = H
                        
                    alphaNew_1 = alpha0ld_1 + y1 * y2 * (alpha0ld_2 - alphaNew_2)
                    
                    b1New = -1 * E1 - y1 * k11 * (alphaNew_1 - alpha0ld_1) - y2 * k21 * (alphaNew_2 - alpha0ld_2) + self.b
                    b2New = -1 * E2 - y1 * k12 * (alphaNew_1 - alpha0ld_1) - y2 * k22 * (alphaNew_2 - alpha0ld_2) + self.b
                    
                    if (alphaNew_1 > 0) and (alphaNew_1 < self.C):
                        bNew = b1New
                    elif (alphaNew_2 > 0) and (alphaNew_2 < self.C):
                        bNew = b2New
                    else:
                        bNew = (b1New + b2New) / 2
                    
                    # Update
                    self.alpha[i] = alphaNew_1
                    self.alpha[j] = alphaNew_2
                    self.b = bNew
                    
                    self.E[i] = self.calc_Ei(i)
                    self.E[j] = self.calc_Ei(j)
                    
                    if math.fabs(alphaNew_2 - alpha0ld_2) >= 0.00001:
                        paramterChanged += 1
                        
                print("iter: %d i : %d, pairs chaged %d " % (iterStep, i, paramterChanged))
            
        for i in range(self.m):
            if self.alpha[i] > 0:
                self.supportVecIndex.append(i)
                
    def calc_singleKernal(self, x1, x2):
        """
        单独计算核函数
        """
        result = (x1 - x2) * (x1 - x2).T
        result = np.exp(-1 * result / (2 * self.sigma ** 2))
        
        return np.exp(result)
    
    def predict(self, x):
        """
        预测
        """
        result = 0
        for i in self.supportVecIndex:
            tmp = self.calc_singleKernal(self.trainDataMat[i, :], np.mat(x))
            result += self.alpha[i] * self.trainLabelMat[i] * tmp
            
        result += self.b
        return np.sign(result)
    
    def test(self, testDataList, testLabelList):
        """
        测试
        """
        
        error_count = 0
        y_pred = []
        for i in range(len(testDataList)):
            result = self.predict(testDataList[i])
            y_pred.append(result)
            if result != testLabelList[i]:
                error_count += 1
        
        acc = 1 - error_count / len(testDataList)
        return acc, y_pred
    
    

In [22]:
X = [[1, 2], [2, 3], [3, 3], [2, 1], [3, 2]]
y = [1, 1, 1, -1, -1]
svm = SVM(X, y, 10, 0.5, 0.001)
svm.train()

construct the kernel:  0 5
iter: 0: 100
iter: 1 i : 0, pairs chaged 1 
iter: 1 i : 3, pairs chaged 1 
iter: 1: 100
iter: 2 i : 0, pairs chaged 0 
iter: 2 i : 3, pairs chaged 0 


In [23]:
y_pred = svm.predict([[1, 2]])


In [24]:
type(y_pred)

numpy.matrix

In [25]:
y_pred

matrix([[1.]])